In [244]:
import random
import re
from urllib.parse import urlparse, parse_qs, parse_qsl, quote

import pandas as pd
import requests

from tqdm import tqdm

from bs4 import BeautifulSoup
import bs4

URL_ROOT = 'https://en.wiktionary.org'

soups = {}
def get_soup(path):
    if path in soups:
        return soups[path]
    
    res = requests.get(f"{URL_ROOT}{path}")
    soup = BeautifulSoup(res.text)
    soups[path] = soup
    return soup

# e.g. get_category_pages("/wiki/Category:South_Levantine_Arabic_terms_with_usage_examples")
def get_category_pages(start_path):
    paths = [start_path]    
    next_path = start_path
    soup = get_soup(next_path)
    next_link = soup.find('a', string='next page')
    while next_link:
        next_path = next_link['href']
        paths.append(next_path)
        soup = get_soup(next_path)
        next_link = soup.find('a', string='next page')
    return paths

# e.g. get_lemma_pages("/wiki/Category:South_Levantine_Arabic_terms_with_usage_examples")
def get_lemma_pages(path):
    soup = get_soup(path)
    return [{'href': a['href'], 'title': a['title']} for a in soup.css.select('div#mw-pages li a')]

In [245]:
pd.set_option('display.max_colwidth', 250)

In [246]:
# Category:Armenian_terms_with_usage_examples
# Category:Old_Armenian_terms_with_usage_examples

lps = ([lp for cp in get_category_pages('/wiki/Category:Armenian_terms_with_usage_examples')
    for lp in get_lemma_pages(cp)])
lemmas = pd.DataFrame(lps)
lemmas

,href,title
0,/wiki/%D5%A1,ա
1,/wiki/%D5%A1%CC%88,ա̈
2,/wiki/%D5%A1%CC%8A,ա̊
3,/wiki/%D5%A1%D5%A2%D5%A5%D5%A9%D5%A1%D5%BD%D5%B8%D6%82%D5%B6%D5%AF,աբեթասունկ
4,/wiki/%D5%A1%D5%A3%D5%BC%D5%A1%D5%BE,ագռավ
...,...,...
1788,/wiki/%D5%96%D6%80%D5%B8%D6%82%D5%B6%D5%A6%D5%A5,Ֆրունզե
1789,/wiki/%D5%9B,՛
1790,/wiki/%D5%9E,՞
1791,/wiki/%D6%89,։


In [247]:
def getLanguageSections(soup):
    headings = soup.select('div#bodyContent div.mw-heading2')
    
    heading_map = {}

    for heading in headings:
        language = heading.find('h2').text
        content = []
        for sib in heading.next_siblings:
            if sib == '\n':
                continue
            if type(sib) == bs4.element.Comment:
                continue
            if sib in headings:
                break
            content.append(sib)
        heading_map[language] = content
    return heading_map

In [248]:
def getHeadingLevel(cs):
    levels = [c for c in cs if c != 'mw-heading']
    if len(levels) > 0:
        return levels[0]

def getSubSections(section):
    results = []
    result = {}
    current_header = None
    current_subsection = bs4.Tag(name='div')
    top_heading_level = None
    for tag in section:
        classes = tag.attrs.get('class', [])
        if tag.name == 'div' and 'mw-heading' in classes:
            heading_level = getHeadingLevel(classes)
            if not top_heading_level:
                top_heading_level = heading_level
            elif heading_level == top_heading_level and result:
                # the first time you encounter the top-level heading again...
                if len(result) > 1: # this means there were other headings in between
                    results.append(result)
                    result = {}
                else: # consecutive top-level headings means flat hierarchy
                    top_heading_level = '_' # non-existent heading
            if current_header:
                result[current_header] = current_subsection
            current_header = tag.text.replace('[edit]', '')
            current_subsection = bs4.Tag(name='div')
        else:
            # if not heading, add to current heading's subsection 
            current_subsection.append(tag)
    if current_header:
        result[current_header] = current_subsection
    results.append(result)
    return results

In [249]:
len({'a': 1})

1

In [250]:
entries = {}

for title, href in tqdm([row[['title','href']].to_list() for _, row in lemmas.sample(100).iterrows()]):
#for title, href in tqdm([row[['title','href']].to_list() for _, row in lemmas[lemmas.title.isin(['աման', 'փիս', '-անք', 'համ'])].iterrows()]):
    sections = getLanguageSections(get_soup(href))
    if 'Armenian' not in sections:
        continue
    entries[title] = getSubSections(sections['Armenian'])

100%|██████████| 100/100 [01:43<00:00,  1.03s/it]


In [278]:
results = pd.DataFrame([dict(title=k, idx=i+1, **r) for k,v in entries.items() for i,r in enumerate(v)]).set_index(['title', 'idx'])

In [279]:
results

,,Etymology,Pronunciation,Noun,Declension,Postposition,References,Derived terms,Proper noun,See also,Further reading,...,Suffix,Etymology 2,Related terms,Descendants,Phrase,Antonyms,Conjunction,Particle,Conjugation,Pronoun
title,idx,,,,,,,,,,,,,,,,,,,,,
հանդերձ,1,"[[[Learned borrowing], from , [<a class=""extiw"" href=""https://en.wikipedia.org/wiki/Classical_Armenian"" title=""w:Classical Armenian"">Old Armenian</a>], , [<a class=""mw-selflink-fragment"" href=""#Old_Armenian"">հանդերձ</a>], , [(], [handerj], [)]...","[[[], \n, [<span class=""ib-brac qualifier-brac"">(</span>, <span class=""ib-content qualifier-content""><span class=""usage-label-accent""><a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian"">Eastern Armeni...","[[[<strong class=""Armn headword"" lang=""hy"">հանդերձ</strong>, , <a href=""/wiki/Wiktionary:Armenian_transliteration"" title=""Wiktionary:Armenian transliteration"">•</a>, (, <span class=""headword-tr tr Latn"" dir=""ltr"" lang=""hy-Latn"">handerj</span>, ...","[[\n, [<small> <i>i</i>-type, inanimate</small>, (, <a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian"">Eastern Armenian</a>, )], \n, [\n, <table class=""inflection-table"" style=""background:#F9F9F9;te...","[[[<strong class=""Armn headword"" lang=""hy"">հանդերձ</strong>, , <a href=""/wiki/Wiktionary:Armenian_transliteration"" title=""Wiktionary:Armenian transliteration"">•</a>, (, <span class=""headword-tr tr Latn"" dir=""ltr"" lang=""hy-Latn"">handerj</span>, ...","[[[<span class=""citation-whole""><span class=""cited-source""><a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eduard_Aghayan"" title=""w:Eduard Aghayan"">Aġayan, Ēduard</a> (<span class=""None"" lang=""und"">1976</span>) “<a class=""external text"" href...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
զորախումբ,1,"[[From , [<a href=""/wiki/%D5%A6%D5%B8%D6%80%D6%84#Armenian"" title=""զորք"">զոր-</a>], , [(], [zor-], [)], +‎ , [<a href=""/wiki/-%D5%A1-#Armenian"" title=""-ա-"">-ա-</a>], , [(], [-a-], [)], +‎ , [<a href=""/wiki/%D5%AD%D5%B8%D6%82%D5%B4%D5%A2#Armen...","[[[], \n, [<span class=""ib-brac qualifier-brac"">(</span>, <span class=""ib-content qualifier-content""><span class=""usage-label-accent""><a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian"">Eastern Armeni...","[[[<strong class=""Armn headword"" lang=""hy"">զորախումբ</strong>, , <a href=""/wiki/Wiktionary:Armenian_transliteration"" title=""Wiktionary:Armenian transliteration"">•</a>, (, <span class=""headword-tr tr Latn"" dir=""ltr"" lang=""hy-Latn"">zoraxumb</span...","[[\n, [<small> <i>i</i>-type, inanimate</small>, (, <a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian"">Eastern Armenian</a>, )], \n, [\n, <table class=""inflection-table"" style=""background:#F9F9F9;te...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
գնացք,1,"[[[Learned borrowing], from , [<a class=""extiw"" href=""https://en.wikipedia.org/wiki/Classical_Armenian"" title=""w:Classical Armenian"">Old Armenian</a>], , [<a class=""mw-selflink-fragment"" href=""#Old_Armenian"">գնացք</a>], , [(], [gnacʻkʻ], [)], ...","[[[], \n, [<span class=""ib-brac qualifier-brac"">(</span>, <span class=""ib-content qualifier-content""><span class=""usage-label-accent""><a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian"">Eastern Armeni...","[[[<strong class=""Armn headword"" lang=""hy"">գնացք</strong>, , <a href=""/wiki/Wiktionary:Armenian_transliteration"" title=""Wiktionary:Armenian transliteration"">•</a>, (, <span class=""headword-tr tr Latn"" dir=""ltr"" lang=""hy-Latn"">gnacʻkʻ</span>, )]...","[[\n, [<small> <i>i</i>-type, inanimate</small>, (, <a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian"">Eastern Armenian</a>

In [280]:
columns_by_freq = results.count().sort_values(ascending=False)
results = results[columns_by_freq.index] #[lambda x:x['Verb'].notna()]
columns_by_freq.to_frame().style.bar()

,0
Pronunciation,99
Etymology,90
Declension,82
Noun,62
Derived terms,28
Adjective,23
Alternative forms,23
References,19
Verb,11
Related terms,10


In [281]:
def process_pronunciation(tag):
    if type(tag) != bs4.Tag:
        return tag
    
    return '\n'.join([li.text for li in tag.select('li') if not 'Audio' in li.text and not li.text == ''])

results['Pronunciation'] = results['Pronunciation'].apply(process_pronunciation)

In [282]:
def process_basic_text(tag):
    if type(tag) != bs4.Tag:
        return tag
    
    return tag.text.strip()


results['Etymology'] = results['Etymology'].apply(process_basic_text)
results['Etymology'] = results['Etymology 1'].apply(process_basic_text).where(results['Etymology 1'].notna(), results['Etymology'])
results['Etymology'] = results['Etymology 2'].apply(process_basic_text).where(results['Etymology 2'].notna(), results['Etymology'])

In [283]:
results.sample(10)

,,Pronunciation,Etymology,Declension,Noun,Derived terms,Adjective,Alternative forms,References,Verb,Related terms,...,Antonyms,Etymology 1,Etymology 2,Suffix,Phrase,Postposition,Conjunction,Particle,Punctuation mark,Pronoun
title,idx,,,,,,,,,,,,,,,,,,,,,
նույնպես,1,"(Eastern Armenian) IPA(key): /ˈnujnpes/, [nújnpes]\n(Western Armenian) IPA(key): /ˈnujnbes/, [nújnbes]\nHyphenation: նույ‧պես",From Old Armenian նոյնպէս (noynpēs).,NaN,NaN,NaN,NaN,NaN,"[[[<span class=""citation-whole""><span class=""cited-source""><a class=""extiw"" href=""https://en.wikipedia.org/wiki/hy:%D5%80%D5%B8%D5%BE%D5%B0%D5%A1%D5%B6%D5%B6%D5%A5%D5%BD_%D4%B2%D5%A1%D6%80%D5%BD%D5%A5%D5%B2%D5%B5%D5%A1%D5%B6"" title=""w:hy:Հովհաննե...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
տարբերություն,1,"(Eastern Armenian) IPA(key): /tɑɾbeɾuˈtʰjun/, [tɑɾbeɾut͡sʰjún]\n(Western Armenian) IPA(key): /dɑɾpeɾuˈtʏn/, [dɑɾpʰeɾutʰʏ́n]","From Old Armenian տարբերութիւն (tarberutʻiwn). By surface analysis, տարբեր (tarber) +‎ -ություն (-utʻyun).","[[\n, [<small> <i>n</i>-type, inanimate</small>, (, <a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian"">Eastern Armenian</a>, )], \n, [\n, <table class=""inflection-table"" style=""background:#F9F9F9;te...","[[[<strong class=""Armn headword"" lang=""hy"">տարբերություն</strong>, , <a href=""/wiki/Wiktionary:Armenian_transliteration"" title=""Wiktionary:Armenian transliteration"">•</a>, (, <span class=""headword-tr tr Latn"" dir=""ltr"" lang=""hy-Latn"">tarberutʻy...",NaN,NaN,"[[[<span class=""Armn"" lang=""hy""><a class=""new"" href=""/w/index.php?title=%D5%BF%D5%A1%D6%80%D5%A2%D5%A5%D6%80%D5%B8%D6%82%D5%A9%D5%AB%D6%82%D5%B6&amp;action=edit&amp;redlink=1"" title=""տարբերութիւն (page does not exist)"">տարբերութիւն</a></span>, ,...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
լրտեսություն,1,"(Eastern Armenian) IPA(key): /ləɾtesuˈtʰjun/, [ləɾtesut͡sʰjún]\n(Western Armenian) IPA(key): /ləɾdesuˈtʏn/, [ləɾdesutʰʏ́n]",From Old Armenian լրտեսութիւն (lrtesutʻiwn).,"[[\n, [<small> <i>n</i>-type, inanimate</small>, (, <a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian"">Eastern Armenian</a>, )], \n, [\n, <table class=""inflection-table"" style=""background:#F9F9F9;te...","[[[<strong class=""Armn headword"" lang=""hy"">լրտեսություն</strong>, , <a href=""/wiki/Wiktionary:Armenian_transliteration"" title=""Wiktionary:Armenian transliteration"">•</a>, (, <span class=""headword-tr tr Latn"" dir=""ltr"" lang=""hy-Latn"">lrtesutʻyun...",NaN,NaN,NaN,NaN,NaN,"[[[<span class=""Armn"" lang=""hy""><a href=""/wiki/%D5%AC%D6%80%D5%BF%D5%A5%D5%BD#Armenian"" title=""լրտես"">լրտես</a></span>, , <span class=""mention-gloss-paren annotation-paren"">(</span>, <span class=""tr Latn"" lang=""hy-Latn"">lrtes</span>, <span class...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-ի,1,"(Eastern Armenian) IPA(key): /i/, [i]\n(Western Armenian) IPA(key): /i/, [i]",From Old Armenian -ի (-i).,NaN,NaN,"[[[<div class=""CategoryTreeItem""><span class=""CategoryTreeBullet""><a aria-expanded=""false"" class=""CategoryTreeToggle"" data-ct-title=""Armenian_terms_suffixed_with_-ի""></a> </span> <a href=""/wiki/Category:Armenian_terms_suffixed_with_-%D5%AB"" title...",NaN,NaN,NaN,NaN,NaN,...,NaN,"[[From , [<a class=""extiw"" href=""https://en.wikipedia.org/wiki/Classical_Armenian"" title=""w:Classical Armenian"">Old Armenian</a>], , [<a class=""mw-selflink-fragment"" href=""#Old_Armenian"">-ի</a>], , [(], [-i], [)], .\n]]",NaN,"[[[<strong class=""Armn headword"" lang=""hy"">-ի</strong>, , <a href=""/wiki/Wiktionary:Armenian_transliteration"" title=""Wiktionary:Armenian transliteration"">•</a>, (, <span class=""headword-tr tr Latn"" dir=""ltr"" lang=""hy-Latn"">-i</span>, )], \n], [...",NaN,NaN,NaN,NaN,NaN,NaN
Հայաստան,1,"(Eastern Armenian) IPA(key): /hɑjɑsˈtɑn/, [hɑjɑstɑ́n]\n(Western Armenian) IPA(key): /hɑjɑsˈdɑn/, [hɑjɑstɑ́n]","From Old Armenian Հայաստան (Hayastan). B